# Figures for 2019 ADAP Proposal

* Magnitude-surface brightness with notable large galaxies labeled
* Show color-magnitude space with Tractor mag/color and custom (aperture) mag/color.
* Need for this analysis:
  - Where does the Tractor pipeline break down?
  - Which galaxies require custom analysis?

In [ ]:
import os, warnings, pdb
from glob import glob
import numpy as np
import numpy.ma as ma
import pandas as pd
from astrometry.util.fits import fits_table

import fitsio
from astropy.table import Table, Column, vstack, hstack

In [ ]:
import corner
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
import LSLGA.io
import LSLGA.NLSA
import LSLGA.qa
import LSLGA.ellipse

In [ ]:
import seaborn as sns
sns.set(style='ticks', font_scale=1.8, palette='Set2')
%matplotlib inline

In [ ]:
LSLGAdir = os.getenv('LSLGA_DIR')
#adapdir = os.path.join(os.getenv('HOME'), 'repos', 'papers', 'grants', 'nasa-adap-2019')
adapdir = os.path.join(LSLGAdir, 'science', 'proposals', 'nasa-adap-2019')

In [ ]:
PIXSCALE = 0.262

In [ ]:
seed = 1
rand = np.random.RandomState(seed)

### Ellipse-fitting NGC5322

In [ ]:
tt = Table.read('/Users/ioannis/research/projects/LSLGA/NLSA/8/1/184/NGC5322/NGC5322-tractor.fits')
tt = tt[np.argsort(tt['shapedev_r'])[::-1]][0]
cols = ['ra', 'dec', 'type', 'shapedev_r',]
cols += ['flux_{}'.format(bb) for bb in ('g', 'r', 'z', 'w1', 'w2', 'w3', 'w4')]
cols += ['flux_ivar_{}'.format(bb) for bb in ('g', 'r', 'z', 'w1', 'w2', 'w3', 'w4')]
tt[cols]

In [ ]:
bands = ('FUV', 'NUV', 'g', 'r', 'z', 'W1', 'W2', 'W3', 'W4')
onegal = LSLGA.NLSA.read_nlsa_parent(montage=True, first=1, last=1)[0]
galaxy, galaxydir = LSLGA.NLSA.get_galaxy_galaxydir(onegal)
#%time LSLGA.ellipse.LSLGA_ellipse(onegal, galaxy=gal, galaxydir=galdir, 
#                                  maxsma=1500, verbose=True, nproc=4)
data = LSLGA.io.read_multiband(galaxy, galaxydir, band=bands)

In [ ]:
#pixscale, wise_pixscale = 2.75, galex_pixscale = 1.5
pixscale = 0.262
filt2pixscalefactor = {'g': 1.0, 'r': 1.0, 'z': 1.0, 'g_pixscale': pixscale,
                       'r_pixscale': pixscale, 'z_pixscale': pixscale}
factor = pixscale / data['galex_pixscale']
filt2pixscalefactor.update({'FUV': factor, 'NUV': factor, 'FUV_pixscale': data['galex_pixscale'],
                            'NUV_pixscale': data['galex_pixscale']})
        
factor = pixscale / data['unwise_pixscale']
filt2pixscalefactor.update({'W1': factor, 'W2': factor, 'W3': factor, 'W4': factor,
                            'W1_pixscale': data['unwise_pixscale'], 'W2_pixscale': data['unwise_pixscale'],
                            'W3_pixscale': data['unwise_pixscale'], 'W4_pixscale': data['unwise_pixscale']})

In [ ]:
phot = []
radius = 1400 # [grz pixels]
for filt in bands:
    pixscale = filt2pixscalefactor['{}_pixscale'.format(filt)]
    pixscalefactor = filt2pixscalefactor[filt]
    img = ma.getdata(data['{}_masked'.format(filt)]) # [nanomaggies/arcsec2]
    mask = ma.getmask(data['{}_masked'.format(filt)])
    x0, y0 = np.array(img.shape) // 2 * pixscalefactor
    rad = radius * pixscalefactor
    print(filt, img.shape, x0, y0, rad)
    phot1 = LSLGA.ellipse.apphot_one(img, mask, 0.0, x0, y0, rad, rad, pixscale)
    phot.append(phot1)

In [ ]:
phot

In [ ]:
stop

In [ ]:
plt.imshow(np.log10(data['W4']), origin='lower')

In [ ]:
ell = LSLGA.io.read_ellipsefit(gal, galdir, verbose=True)
pngfile = os.path.join(adapdir, 'ngc5322-sbprofile.png')
LSLGA.qa.display_ellipse_sbprofile(ell, png=pngfile, verbose=True)
pngfile = os.path.join(adapdir, 'ngc5322-cog.png')
LSLGA.qa_curveofgrowth(ell, png=pngfile, verbose=True)

### Read dr8 catalogs.

In [ ]:
lslga = LSLGA.io.read_parent(verbose=True)
lslga[:3]

In [ ]:
#cat = LSLGA.NLSA.read_nlsa_parent()
camera = '90prime-mosaic'
samplefile = os.path.join(LSLGA.NLSA.sample_dir(), 'NLSA-{}-v1.2.fits'.format(camera))
cat = Table(fitsio.read(samplefile, lower=True))
cat

In [ ]:
cat[cat['ref_cat'] == 'L2']

In [ ]:
ww = np.where(['NGC' in gg or 'UGC' in gg for gg in cat['galaxy']])[0]
#cat[ww]

In [ ]:
ss = LSLGA.NLSA.read_nlsa_parent(proposal=True)
_ = plt.hist(ss['SB'], bins=50)
ss['SB'].data

In [ ]:
def ngc3938_ccdpos():
    from astrometry.util.fits import fits_table
    from legacypipe.survey import LegacySurveyData
    def get_ccds():
        survey = LegacySurveyData()
        galaxy = 'NGC3938'
        galaxydir = os.path.join(os.getenv('LSLGA_DIR'), 'NLSA', '8', '0', '85', galaxy)
        ccdsfile = os.path.join(galaxydir, '{}-ccds.fits'.format(galaxy))
        ccds = survey.cleanup_ccds_table(fits_table(ccdsfile))
        return ccds
    ccds = ngc3938_ccdpos()
    import LSLGA.NLSA
    import LSLGA.qa
    onegal = LSLGA.NLSA.read_nlsa_parent(montage=True, first=0, last=0)[0]
    LSLGA.qa.display_ccdpos(onegal, ccds, png='ccdpos-ngc3938.png', radius=196)

In [ ]:
ww = ['NGC5322' in gg.strip() for gg in cat['galaxy']]
cat[ww]

In [ ]:
#ww = ['3938' in gg.strip() for gg in cat['galaxy']]
#from astrometry.libkd.spherematch import match_radec
#m1, m2, d12 = match_radec(cat['ra'], cat['dec'], 178.205851, 44.120774, 3/3600.0, nearest=True)
#cat[m1]

In [ ]:
def ngc3938_montage():
    montagefile = os.path.join(adapdir, 'ngc3938-montage.png')
    gal = 'NGC3938'
    galdir = os.path.join(os.getenv('LSLGA_DIR'), 'NLSA', '8', '0', '85', gal)
    infiles = []
    for bands in ('FUVNUV', 'grz', 'image-W1W2'):
        for imtype in ('image-FUVNUV', 'pipeline-image-grz', 'image-W1W2'):
            infiles.append(os.path.join(galdir, '{}-{}.jpg'.format(gal, bands)))
    ncol = 4
    nrow = 3
    
    cmd = 'montage -bordercolor white -borderwidth 1 -tile {}x{} -geometry 512x512 '.format(ncol, nrow)
    cmd = cmd+' '.join([inff for inff in infiles])
    cmd = cmd+' {}'.format(montagefile)
    print(cmd)
    os.system(cmd)        
    print('Writing {}'.format(montagefile))

In [ ]:
#ngc3938_montage()

In [ ]:
def sample_montage():
    """Show a random sampling of galaxies."""
    montagefile = os.path.join(adapdir, 'sample-montage.png')

    ss = LSLGA.NLSA.read_nlsa_parent(proposal=True)
    #these = rand.choice(len(ss), 35, replace=False)
    #ss = ss[these]
    ss = ss[np.argsort(ss['SB'])]
        
    gal, galdir = LSLGA.NLSA.get_galaxy_galaxydir(ss)
    infiles = []
    for gal1, galdir1 in zip(gal, galdir):
        infile1 = os.path.join(galdir1, '{}-pipeline-image-grz.jpg'.format(gal1))
        if not os.path.isfile(infile1):
            infile1 = os.path.join(adapdir, 'smiley.jpg')
        infiles.append(infile1)
    ncol = 7
    nrow = 5
    
    #cmd = 'montage -tile {}x{} -geometry 128x128 '.format(ncol, nrow)
    cmd = 'montage -bordercolor white -borderwidth 1 -tile {}x{} -geometry 256x256 '.format(ncol, nrow)
    cmd = cmd+' '.join([inff for inff in infiles])
    cmd = cmd+' {}'.format(montagefile)
    print(cmd)
    os.system(cmd)        
    print('Writing {}'.format(montagefile))

In [ ]:
sample_montage()

In [ ]:
stop

In [ ]:
def qa_rmag_sb(cat, frac=None, png=None):
    """Surface brightness vs r-band magnitude.
    
    """
    xlim, ylim = (12, 20), (28, 17)
    #xlim, ylim = (12, 22), (18, 28)
    rmag, sb, reff = cat['rmag'], cat['sb'], cat['reff']
    
    print('Hack to remove sources that hit their optimization limit.')
    for factor in (0.788, 0.7999, 0.763, 0.813):
        rcut = 64*0.262*factor
        toss = (reff > rcut-0.01) * (reff < rcut+0.01)
        print(rcut, np.sum(toss))
        rmag, sb, reff = rmag[~toss], sb[~toss], reff[~toss]

    if frac:
        nn = np.round(len(rmag) * frac).astype(int)
        sel = rand.choice(len(rmag), nn, replace=False)
        print('Subselecting {} objects.'.format(nn))
        rmag = rmag[sel]
        sb = sb[sel]
    
    fig, ax = plt.subplots(figsize=(8, 6))
    corner.hist2d(rmag, sb, levels=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99],
                  bins=100, smooth=True, color='tan', # mpl.cm.get_cmap('viridis'),
                  plot_density=True, fill_contours=True,
                  data_kwargs={'color': 'navy', 'alpha': 0.2, 'ms': 3},
                  contour_kwargs={'colors': 'k'}
                 )
           
    # overplot lines of constant size
    magarr = np.linspace(xlim[0], xlim[1], 50)
    for size, xpos, ypos in zip((0.3, 1, 3, 10, 30, 100),
                                (None, None, None, 12.2, 12.2, 12.2),
                                (18.0, 18.0, 18.0, None, None, None)):
        sbarr = magarr + 2.5 * np.log10(2 * np.pi * size**2)
        ax.plot(magarr, sbarr, color='k', lw=2, ls='--', alpha=0.8)
        
        if ypos:
            xpos = ypos - 2.5 * np.log10(2 * np.pi * size**2)
        if xpos:
            ypos = xpos + 2.5 * np.log10(2 * np.pi * size**2)
            
        ax.text(xpos, ypos, r'$r_{{50}}={0:g}^{{\prime\prime}}$'.format(size), 
                ha='left', va='center', rotation=-30, fontsize=16, alpha=1.0)

    check = np.where( (cat['sb'] > cat['rmag']+2.5*np.log10(2*np.pi*10**2)) * (cat['rmag'] < 18) )[0]
    print(len(check), len(cat))
    #ax.scatter(cat['rmag'][check], cat['sb'][check], color='blue', marker='s', s=10)
    #for ii, cc in enumerate(check[:30]):
    #    print('gal{}'.format(ii), cat['ra'][cc], cat['dec'][cc], cat['reff'][cc], 
    #          cat['rmag'][cc], cat['type'][cc], cat['brightblob'][cc] & 0x2)
    
    lgal = cat['ref_cat'] == 'L2'
    #ax.scatter(cat['rmag'][lgal], cat['sb'][lgal], marker='s', edgecolors='k', s=30, 
    #           alpha=0.5, lw=1, color='firebrick')
    
    ingc = np.where(['NGC' in gg or 'UGC' in gg for gg in cat['galaxy']])[0]
    #ax.scatter(cat['rmag'][ingc], cat['sb'][ingc], marker='s', edgecolors='k', s=30, 
    #           alpha=0.5, lw=1, color='firebrick')
    
    ingc = np.where(['NGC' in gg for gg in cat['galaxy']])[0]
    #print(cat[ingc])
    #for ii in ingc:
    #    ax.text(cat['rmag'][ii], cat['sb'][ii], cat['galaxy'][ii], 
    #            ha='left', va='bottom', fontsize=20)
    
    # labels and all that
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    #ax.xmargins(0)
    ax.set_xlabel(r'$m_{r}$ (AB mag)')
    ax.set_ylabel(r'$\mu_{50,r}$ (AB mag arcsec$^{-2}$)')
    
    fig.subplots_adjust(bottom=0.15, top=0.95, right=0.95)
    
    if png:
        pngfile = os.path.join(adapdir, png)
        print('Writing {}'.format(pngfile))
        fig.savefig(pngfile)

In [ ]:
qa_rmag_sb(cat, png='sb-mag.png', frac=0.5)

In [ ]:
stop

In [ ]:
stop

In [ ]:
#_ = plt.hist(cat['reff'], bins=100, range=(12, 14))
#plt.xlabel('fracdev*rdev + (1-fracdev)*rexp [arcsec]')
#plt.ylabel('Number')
#plt.axvline(x=64*0.788*0.262, color='k')
#plt.axvline(x=64*0.7999*0.262, color='k')
#plt.axvline(x=64*0.763*0.262, color='k')
#plt.axvline(x=64*0.813*0.262, color='k')
#cat[cat['reff'].argmax()]

In [ ]:
_ = plt.hist(cat['sb'], bins=100)

In [ ]:
def multiwavelength_montage():
    montagefile = os.path.join(adapdir, 'multiwavelength-montage.png')

    ss = LSLGA.NLSA.read_nlsa_parent(proposal=True)
    gal, galdir = LSLGA.NLSA.get_galaxy_galaxydir(ss)
    infiles = []
    for bands in ('image-FUVNUV', 'pipeline-image-grz', 'image-W1W2'):
        for gal1, galdir1 in zip(gal, galdir):
            infiles.append(os.path.join(galdir1, '{}-{}.jpg'.format(gal1, bands)))
    ncol = 4
    nrow = 3
    
    cmd = 'montage -bordercolor white -borderwidth 1 -tile {}x{} -geometry 512x512 '.format(ncol, nrow)
    cmd = cmd+' '.join([inff for inff in infiles])
    cmd = cmd+' {}'.format(montagefile)
    print(cmd)
    os.system(cmd)        
    print('Writing {}'.format(montagefile))

In [ ]:
multiwavelength_montage()

In [ ]:
stop

In [ ]:
ww = allcat['FLUX_R'] > 10**(0.4*(22.5-23))
_ = plt.hist(22.5-2.5*np.log10(allcat['FLUX_R'][ww]), bins=20, normed=True)
_ = plt.hist(22.5-2.5*np.log10(lslgacat['FLUX_R']), bins=20, normed=True)